In [3]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import tqdm

In [4]:
with open('all_children.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [5]:
childrenList = pd.DataFrame()

def getChildInfo(number):
    # URL сторінки
    url = f"https://www.msp.gov.ua/children/child.php?n={number}"

    # Виконання HTTP-запиту
    response = requests.get(url)

    # Перевірка статусу відповіді
    if response.status_code == 200:
        # get object soup
        soup = BeautifulSoup(response.content, 'html.parser')
        name, age = "no name", "no age"
        name_age = soup.find(class_='passport__title-name').text #get name & age
        name = name_age.split(',')[0] #get name
        age = (name_age.split(' ')[1]) #get age
        child_id = soup.find(class_='passport__title-number').text.split('№')[1] #get id

        # get block passport__content with information with child
        passport_content = soup.find('div', class_='row passport__content')
        gender = "uknown"
        gender_element = passport_content.find('span', class_='passport__content-title', string="Стать:")
        if gender_element:
            gender = gender_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Стать не знайдено.")
        
        # get region
        region = "uknown"
        region_element = passport_content.find('span', class_='passport__content-title', string="Область:")
        if region_element:
            region = region_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Область не знайдено.")
        
        #get disablement status
        disablement = "uknown"
        disablement_element = passport_content.find('span', class_='passport__content-title', string="Наявність інвалідності:")
        if disablement_element:
            disablement = disablement_element.find_next('span', class_='passport__content-inner').text.strip()
            if disablement == 'є':
                disablement = True
            else:
                disablement = False

        else:
            print("Статус інвалідності не знайдено")

        #get form of adoption
        adoptionForm_element = passport_content.find('span', class_='passport__content-title', string="Можлива форма влаштування:")
        if adoptionForm_element:
            adoptionForm = adoptionForm_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Форому всиновлення не знайдено.")

        #place where child is now
        place_element = "uknown"
        place_element = passport_content.find('span', class_='passport__content-title', string="Куди звертатись:")
        if place_element:
            place = place_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Місце перебування не знайдено.")

        #photo
        photo_element = passport_content.find('a', class_='js-lightBoxGallery')
        if photo_element:
            photo = 'https://www.msp.gov.ua/'+photo_element.find('img').get('src')
        else:
            print("Фотокартка не знайдена.")
        
        #get list of siblings, their names and links to their pages


        # #siblings
        siblings_count = 0
        siblingInfo = {}
        siblings_element = passport_content.find('ul', class_='passport__content-inner passport__content-inner--brother')
        if siblings_element:
            siblings_count = len(siblings_element.findAll('li'))            
            siblings_list_element = siblings_element.findAll('a')
            siblings_list = []
            if len(siblings_list_element) > 0:
                for i in siblings_list_element:
                    siblingInfo = {  # Створюємо новий словник для кожного брата/сестри
                        'link': "https://www.msp.gov.ua" + i.attrs['href'],
                        'name': i.find('div', class_='name').text.strip() if i.find('div', class_='name') else "Unknown",
                        'age': i.find('div', class_='age').text.strip() if i.find('div', class_='age') else "Unknown"
                    }
                    siblings_list.append(siblingInfo)  # Додаємо словник до списку
            else:
                siblings_list = ["no siblings"]  # Якщо список порожній, залишаємо відповідне значення

        else:
             siblings_list = ["no siblings"]

        # checking result
        # print(url,'\nname:',name, '\nage:',age, '\nid:',child_id, '\ngender:', gender, '\nregion: ',region,
        #        '\ndisablement: ', disablement, "\nadoptionForm: ", adoptionForm, "\nplace: ", place, "\nphoto url:", photo,
        #         '\nsiblings_count:', siblings_count, '\n*******\nsiblings_list', siblings_list, "\nsiblings_info", siblingInfo,
        #       '\n--------------------------------------------------','\n--------------------------------------------------')
        
        # forming 
        dfChildren = pd.DataFrame({'name': [name], 
                                   'age': [age], 
                                   'id': [child_id],
                                   'gender':[gender],
                                   'region':[region],
                                   'isablement':[disablement],
                                   'adoptionForm':[adoptionForm],
                                   'place':[place],
                                   'photo':[photo],
                                   'siblings_count':[siblings_count],
                                   'siblings_list':[siblings_list]
                                   })
        
        #save to csv
        # print(dfChildren.siblings_list)
        # dfChildren.to_csv('children_info.csv', mode='a', header=False, index=False, sep='|')
        return dfChildren
        
    
    else:
        print(f"Не вдалося отримати сторінку, статус-код: {response.status_code}")

#перевірка функціоналу
for i in tqdm.tqdm(range(1,len(data))):
     child = getChildInfo(data[i])
     childrenList = pd.concat([childrenList, child], ignore_index=True)
     
    # getChildInfo('388104')

childrenList.to_csv('children_info.csv', mode='w', header=True, index=False, sep='|')

100%|██████████| 13924/13924 [1:13:50<00:00,  3.14it/s]


In [9]:
childrenList.to_csv('children_info.csv', mode='w', header=True, index=False, sep='|')

In [11]:
childrenList.head()

,name,age,id,gender,region,isablement,adoptionForm,place,photo,siblings_count,siblings_list
0,Валентин,16,6829,чоловіча,Вінницька обл.,False,національне усиновлення;,ССД Жмеринської райдержадміністрації: 04332 5...,https://www.msp.gov.ua/../files/children/02_ch...,2,[{'link': 'https://www.msp.gov.ua/children/chi...
1,Ірина,15,6831,жіноча,Вінницька обл.,False,національне усиновлення; опіка/піклування; при...,ССД Жмеринської райдержадміністрації: 04332 5...,https://www.msp.gov.ua/../files/children/02_ch...,2,[{'link': 'https://www.msp.gov.ua/children/chi...
2,Артем,17,6922,чоловіча,Вінницька обл.,False,національне усиновлення;,ССД Жмеринської райдержадміністрації: 04332 5...,https://www.msp.gov.ua/../files/children/02_ch...,1,[{'link': 'https://www.msp.gov.ua/children/chi...
3,Захар,15,6924,чоловіча,Вінницька обл.,False,національне усиновлення;,ССД Жмеринської райдержадміністрації: 04332 5...,https://www.msp.gov.ua/../files/children/02_ch...,1,[{'link': 'https://www.msp.gov.ua/children/chi...
4,Микита,16,6989,чоловіча,Вінницька обл.,False,національне усиновлення;,ССД Вінницької райдержадміністрації: (0432) 66...,https://www.msp.gov.ua/../files/children/02_ch...,0,[no siblings]
